* **Họ và tên:** Nguyễn Thị Quyên
* **MSSV:** 18021063
* **Topic:** Quora Insincere Questions Classification

# GIỚI THIỆU VỀ BÀI TOÁN
* Quora là một nền tảng để mọi người học hỏi với nhau. Mọi người có thể đăng những câu hỏi và người khác vào chia sẻ, trả lời những thắc mắc đó. Mục đích của bài toán này là để chỉ ra câu hỏi có chân thành hay không. Những câu hỏi không chân thành thường là những câu đưa ra tuyên bố quan điểm của mình hơn là để tìm những câu trả lời có ích: không trung lập, khiêu khích hoặc chê bai, không có căn cứ thực tế và nội dung khiêu dâm.
* Câu hỏi chân thành hay không sẽ được gán nhãn bằng 0 hoặc 1

# EDA

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import matplotlib.pyplot as plt
import seaborn as sb
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk_stopwords = stopwords.words('english')
from keras.preprocessing.sequence import pad_sequences
from sklearn import *

In [ ]:
train_data = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/train.csv')
test_data = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/test.csv')

In [ ]:
print("Train shape :", train_data.shape)
print("Test shape :", test_data.shape)

In [ ]:
train_data.info()

**Nhận xét:** Tập dữ liệu train có 1306122 câu hỏi và tập test chứa 375806 câu hỏi để đánh giá hiệu năng mô hình. Trong đó, không có giá trị null nào và có 3 trường dữ liệu: qid, question_text, target.
- qid : id của câu hỏi, mỗi câu hỏi có 1 id riêng phân biệt
- question_text : nội dung câu hỏi ở dạng text
- target: gồm 2 nhãn 0 và 1. Nhãn 0 là sincere là câu hỏi chân thành, nhãn 1 là insincere là câu hỏi không chân thành

**Biểu đồ biểu diễn sự phân bố của dữ liệu**

In [ ]:
sb.countplot(train_data['target'])

In [ ]:
val_counts = train_data['target'].value_counts()
val_counts

 **TARGET DISTRIBUTION**


In [ ]:
sincere_percent= (val_counts[0] / val_counts.sum())* 100
insincere_percent= 100 - sincere_percent

In [ ]:
print(sincere_percent, insincere_percent)

In [ ]:
labels = 'Sincere', 'Insincere'
sizes = [sincere_percent, insincere_percent]
plt.pie(sizes, labels=labels,
autopct='%1.1f%%', shadow=True, startangle=140)
plt.axis('equal')
plt.show()

**Nhận xét:** Số lượng câu hỏi sincerce:insincere có sự chênh lệch rất lớn: Số lượng câu hỏi chân thành chiếm đến gần 94% tập dữ liệu trong khi số lượng câu hỏi không chân thành chỉ chiếm 6.2%.
 
 -> Dữ liệu này mất cân bằng rất lớn
 
 -> Cần áp dụng các phương pháp cân bằng dữ liệu
 
 -> F1-score sẽ được sử dụng để đánh giá hiệu năng của mô hình. F1 score là độ cân bằng đồng đều giữa precision và recall

# TIỀN XỬ LÝ

**Dữ liệu này ở dạng text. Ta cần chuyển sang dạng vector để mô hình có thể học được. Trước hết ta xử lý nhiễu dữ liệu.**

Ta sẽ loại bỏ noise của dữ liệu theo các bước sau:
*     ***clean_punct()***: loại bỏ đi các kí tự đặc biệt có trong câu hỏi
*     ***clean_numbers()***: xóa đi các chữ số
*     ***clean_misspells()***: sửa các từ tiếng anh viết tắt hoặc viết sai
*     ***clean_stopwords()***: loại bỏ các stopwords như 'a', 'the', 'does',... 

In [ ]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', 
        '•', '~', '@', '£', '·', '_', '{', '}', '©', '^', '®', '`', '<', '→', '°', '€', '™', '›', '♥', '←', '×', '§', '″', '′', 
        '█', '…', '“', '★', '”', '–', '●', '►', '−', '¢', '¬', '░', '¡', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', 
        '—', '‹', '─', '▒', '：', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', '¯', '♦', '¤', '▲', '¸', '⋅', '‘', '∞', 
        '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '・', '╦', '╣', '╔', '╗', '▬', '❤', '≤', '‡', '√', '◄', '━', 
        '⇒', '▶', '≥', '╝', '♡', '◊', '。', '✈', '≡', '☺', '✔', '↵', '≈', '✓', '♣', '☎', '℃', '◦', '└', '‟', '～', '！', '○', 
        '◆', '№', '♠', '▌', '✿', '▸', '⁄', '□', '❖', '✦', '．', '÷', '｜', '┃', '／', '￥', '╠', '↩', '✭', '▐', '☼', '☻', '┐', 
        '├', '«', '∼', '┌', '℉', '☮', '฿', '≦', '♬', '✧', '〉', '－', '⌂', '✖', '･', '◕', '※', '‖', '◀', '‰', '\x97', '↺', 
        '∆', '┘', '┬', '╬', '،', '⌘', '⊂', '＞', '〈', '⎙', '？', '☠', '⇐', '▫', '∗', '∈', '≠', '♀', '♔', '˚', '℗', '┗', '＊', 
        '┼', '❀', '＆', '∩', '♂', '‿', '∑', '‣', '➜', '┛', '⇓', '☯', '⊖', '☀', '┳', '；', '∇', '⇑', '✰', '◇', '♯', '☞', '´', 
        '↔', '┏', '｡', '◘', '∂', '✌', '♭', '┣', '┴', '┓', '✨', '\xa0', '˜', '❥', '┫', '℠', '✒', '［', '∫', '\x93', '≧', '］', 
        '\x94', '∀', '♛', '\x96', '∨', '◎', '↻', '⇩', '＜', '≫', '✩', '✪', '♕', '؟', '₤', '☛', '╮', '␊', '＋', '┈', '％', 
        '╋', '▽', '⇨', '┻', '⊗', '￡', '।', '▂', '✯', '▇', '＿', '➤', '✞', '＝', '▷', '△', '◙', '▅', '✝', '∧', '␉', '☭', '┊', '╯', '☾', '➔', '∴', '\x92', '▃', '↳', '＾', '׳', '➢', '╭', '➡', '＠', '⊙', '☢', '˝', '∏', '„', '∥', '❝', '☐', 
        '▆', '╱', '⋙', '๏', '☁', '⇔', '▔', '\x91', '➚', '◡', '╰', '\x85', '♢', '˙', '۞', '✘', '✮', '☑', '⋆', 'ⓘ', '❒', 
        '☣', '✉', '⌊', '➠', '∣', '❑', '◢', 'ⓒ', '\x80', '〒', '∕', '▮', '⦿', '✫', '✚', '⋯', '♩', '☂', '❞', '‗', '܂', '☜', 
        '‾', '✜', '╲', '∘', '⟩', '＼', '⟨', '·', '✗', '♚', '∅', 'ⓔ', '◣', '͡', '‛', '❦', '◠', '✄', '❄', '∃', '␣', '≪', '｢', 
        '≅', '◯', '☽', '∎', '｣', '❧', '̅', 'ⓐ', '↘', '⚓', '▣', '˘', '∪', '⇢', '✍', '⊥', '＃', '⎯', '↠', '۩', '☰', '◥', 
        '⊆', '✽', '⚡', '↪', '❁', '☹', '◼', '☃', '◤', '❏', 'ⓢ', '⊱', '➝', '̣', '✡', '∠', '｀', '▴', '┤', '∝', '♏', 'ⓐ', 
        '✎', ';', '␤', '＇', '❣', '✂', '✤', 'ⓞ', '☪', '✴', '⌒', '˛', '♒', '＄', '✶', '▻', 'ⓔ', '◌', '◈', '❚', '❂', '￦', 
        '◉', '╜', '̃', '✱', '╖', '❉', 'ⓡ', '↗', 'ⓣ', '♻', '➽', '׀', '✲', '✬', '☉', '▉', '≒', '☥', '⌐', '♨', '✕', 'ⓝ', 
        '⊰', '❘', '＂', '⇧', '̵', '➪', '▁', '▏', '⊃', 'ⓛ', '‚', '♰', '́', '✏', '⏑', '̶', 'ⓢ', '⩾', '￠', '❍', '≃', '⋰', '♋', 
        '､', '̂', '❋', '✳', 'ⓤ', '╤', '▕', '⌣', '✸', '℮', '⁺', '▨', '╨', 'ⓥ', '♈', '❃', '☝', '✻', '⊇', '≻', '♘', '♞', 
        '◂', '✟', '⌠', '✠', '☚', '✥', '❊', 'ⓒ', '⌈', '❅', 'ⓡ', '♧', 'ⓞ', '▭', '❱', 'ⓣ', '∟', '☕', '♺', '∵', '⍝', 'ⓑ', 
        '✵', '✣', '٭', '♆', 'ⓘ', '∶', '⚜', '◞', '்', '✹', '➥', '↕', '̳', '∷', '✋', '➧', '∋', '̿', 'ͧ', '┅', '⥤', '⬆', '⋱', 
        '☄', '↖', '⋮', '۔', '♌', 'ⓛ', '╕', '♓', '❯', '♍', '▋', '✺', '⭐', '✾', '♊', '➣', '▿', 'ⓑ', '♉', '⏠', '◾', '▹', 
        '⩽', '↦', '╥', '⍵', '⌋', '։', '➨', '∮', '⇥', 'ⓗ', 'ⓓ', '⁻', '⎝', '⌥', '⌉', '◔', '◑', '✼', '♎', '♐', '╪', '⊚', '☒', '⇤', 'ⓜ', '⎠', '◐', '⚠', '╞', '◗', '⎕', 'ⓨ', '☟', 'ⓟ', '♟', '❈', '↬', 'ⓓ', '◻', '♮', '❙', '♤', '∉', '؛', 
        '⁂', 'ⓝ', '־', '♑', '╫', '╓', '╳', '⬅', '☔', '☸', '┄', '╧', '׃', '⎢', '❆', '⋄', '⚫', '̏', '☏', '➞', '͂', '␙', 
        'ⓤ', '◟', '̊', '⚐', '✙', '↙', '̾', '℘', '✷', '⍺', '❌', '⊢', '▵', '✅', 'ⓖ', '☨', '▰', '╡', 'ⓜ', '☤', '∽', '╘', 
        '˹', '↨', '♙', '⬇', '♱', '⌡', '⠀', '╛', '❕', '┉', 'ⓟ', '̀', '♖', 'ⓚ', '┆', '⎜', '◜', '⚾', '⤴', '✇', '╟', '⎛', 
        '☩', '➲', '➟', 'ⓥ', 'ⓗ', '⏝', '◃', '╢', '↯', '✆', '˃', '⍴', '❇', '⚽', '╒', '̸', '♜', '☓', '➳', '⇄', '☬', '⚑', 
        '✐', '⌃', '◅', '▢', '❐', '∊', '☈', '॥', '⎮', '▩', 'ு', '⊹', '‵', '␔', '☊', '➸', '̌', '☿', '⇉', '⊳', '╙', 'ⓦ', 
        '⇣', '｛', '̄', '↝', '⎟', '▍', '❗', '״', '΄', '▞', '◁', '⛄', '⇝', '⎪', '♁', '⇠', '☇', '✊', 'ி', '｝', '⭕', '➘', 
        '⁀', '☙', '❛', '❓', '⟲', '⇀', '≲', 'ⓕ', '⎥', '\u06dd', 'ͤ', '₋', '̱', '̎', '♝', '≳', '▙', '➭', '܀', 'ⓖ', '⇛', '▊', 
        '⇗', '̷', '⇱', '℅', 'ⓧ', '⚛', '̐', '̕', '⇌', '␀', '≌', 'ⓦ', '⊤', '̓', '☦', 'ⓕ', '▜', '➙', 'ⓨ', '⌨', '◮', '☷', 
        '◍', 'ⓚ', '≔', '⏩', '⍳', '℞', '┋', '˻', '▚', '≺', 'ْ', '▟', '➻', '̪', '⏪', '̉', '⎞', '┇', '⍟', '⇪', '▎', '⇦', '␝', 
        '⤷', '≖', '⟶', '♗', '̴', '♄', 'ͨ', '̈', '❜', '̡', '▛', '✁', '➩', 'ா', '˂', '↥', '⏎', '⎷', '̲', '➖', '↲', '⩵', '̗', '❢', 
        '≎', '⚔', '⇇', '̑', '⊿', '̖', '☍', '➹', '⥊', '⁁', '✢']
def clean_punct(x):
  x = str(x)
  for punct in puncts:
    if punct in x:
      x = x.replace(punct, ' ')
    return x

In [ ]:
def clean_numbers(x):
    x = x.translate(str.maketrans('', '', string.digits))
    return x

In [ ]:
misspell_dict = {"I'm": 'I am', "I'm'a": 'I am about to', "I'm'o": 'I am going to',
 "I've": 'I have',"I'll": 'I will',"I'll've": 'I will have',
 "I'd": 'I would',"I'd've": 'I would have','Whatcha': 'What are you',
 "amn't": 'am not',"ain't": 'are not',"aren't": 'are not',
 "'cause": 'because',"can't": 'can not',"can't've": 'can not have',
 "could've": 'could have',"couldn't": 'could not',"couldn't've": 'could not have',
 "daren't": 'dare not',"daresn't": 'dare not',"dasn't": 'dare not',
 "didn't": 'did not','didn’t': 'did not',"don't": 'do not',
 'don’t': 'do not',"doesn't": 'does not',"e'er": 'ever',
 "everyone's": 'everyone is','finna': 'fixing to','gimme': 'give me',
 "gon't": 'go not','gonna': 'going to','gotta': 'got to',
 "hadn't": 'had not',"hadn't've": 'had not have',"hasn't": 'has not',
 "haven't": 'have not',"he've": 'he have',"he's": 'he is',
 "he'll": 'he will',"he'll've": 'he will have',"he'd": 'he would',
 "he'd've": 'he would have',"here's": 'here is',"how're": 'how are',
 "how'd": 'how did',"how'd'y": 'how do you',"how's": 'how is',
 "how'll": 'how will',"isn't": 'is not',"it's": 'it is',
 "'tis": 'it is',"'twas": 'it was',"it'll": 'it will',
 "it'll've": 'it will have',"it'd": 'it would',"it'd've": 'it would have',
 'kinda': 'kind of',"let's": 'let us','luv': 'love',
 "ma'am": 'madam',"may've": 'may have',"mayn't": 'may not',
 "might've": 'might have',"mightn't": 'might not',"mightn't've": 'might not have',
 "must've": 'must have',"mustn't": 'must not',"mustn't've": 'must not have',
 "needn't": 'need not',"needn't've": 'need not have',"ne'er": 'never',
 "o'": 'of',"o'clock": 'of the clock',"ol'": 'old',
 "oughtn't": 'ought not',"oughtn't've": 'ought not have',"o'er": 'over',
 "shan't": 'shall not',"sha'n't": 'shall not',"shalln't": 'shall not',
 "shan't've": 'shall not have',"she's": 'she is',"she'll": 'she will',
 "she'd": 'she would',"she'd've": 'she would have',"should've": 'should have',
 "shouldn't": 'should not',"shouldn't've": 'should not have',"so've": 'so have',
 "so's": 'so is',"somebody's": 'somebody is',"someone's": 'someone is',
 "something's": 'something is','sux': 'sucks',"that're": 'that are',
 "that's": 'that is',"that'll": 'that will',"that'd": 'that would',
 "that'd've": 'that would have','em': 'them',"there're": 'there are',
 "there's": 'there is',"there'll": 'there will',"there'd": 'there would',
 "there'd've": 'there would have',"these're": 'these are',"they're": 'they are',
 "they've": 'they have',"they'll": 'they will',"they'll've": 'they will have',
 "they'd": 'they would',"they'd've": 'they would have',"this's": 'this is',
 "those're": 'those are',"to've": 'to have','wanna': 'want to',
 "wasn't": 'was not',"we're": 'we are',"we've": 'we have',
 "we'll": 'we will',"we'll've": 'we will have',"we'd": 'we would',
 "we'd've": 'we would have',"weren't": 'were not',"what're": 'what are',
 "what'd": 'what did',"what've": 'what have',"what's": 'what is',
 "what'll": 'what will',"what'll've": 'what will have',"when've": 'when have',
 "when's": 'when is',"where're": 'where are',"where'd": 'where did',
 "where've": 'where have',"where's": 'where is',"which's": 'which is',
 "who're": 'who are',"who've": 'who have',"who's": 'who is',
 "who'll": 'who will',"who'll've": 'who will have',"who'd": 'who would',
 "who'd've": 'who would have',"why're": 'why are',"why'd": 'why did',
 "why've": 'why have',"why's": 'why is',"will've": 'will have',
 "won't": 'will not',"won't've": 'will not have',"would've": 'would have',
 "wouldn't": 'would not',"wouldn't've": 'would not have',"y'all": 'you all',
 "y'all're": 'you all are',"y'all've": 'you all have',"y'all'd": 'you all would',
 "y'all'd've": 'you all would have',"you're": 'you are',"you've": 'you have',
 "you'll've": 'you shall have',"you'll": 'you will',"you'd": 'you would',
 "you'd've": 'you would have','jan.': 'january','feb.': 'february',
 'mar.': 'march','apr.': 'april','jun.': 'june',
 'jul.': 'july','aug.': 'august','sep.': 'september',
 'oct.': 'october','nov.': 'november','dec.': 'december',
 'I’m': 'I am','I’m’a': 'I am about to','I’m’o': 'I am going to',
 'I’ve': 'I have','I’ll': 'I will','I’ll’ve': 'I will have',
 'I’d': 'I would','I’d’ve': 'I would have','amn’t': 'am not',
 'ain’t': 'are not','aren’t': 'are not','’cause': 'because',
 'can’t': 'can not','can’t’ve': 'can not have','could’ve': 'could have',
 'couldn’t': 'could not','couldn’t’ve': 'could not have','daren’t': 'dare not',
 'daresn’t': 'dare not','dasn’t': 'dare not','doesn’t': 'does not',
 'e’er': 'ever','everyone’s': 'everyone is','gon’t': 'go not',
 'hadn’t': 'had not','hadn’t’ve': 'had not have',
 'hasn’t': 'has not','haven’t': 'have not',
 'he’ve': 'he have','he’s': 'he is','he’ll': 'he will',
 'he’ll’ve': 'he will have','he’d': 'he would','he’d’ve': 'he would have',
 'here’s': 'here is','how’re': 'how are','how’d': 'how did',
 'how’d’y': 'how do you','how’s': 'how is','how’ll': 'how will',
 'isn’t': 'is not','it’s': 'it is','’tis': 'it is',
 '’twas': 'it was','it’ll': 'it will','it’ll’ve': 'it will have',
 'it’d': 'it would','it’d’ve': 'it would have','let’s': 'let us',
 'ma’am': 'madam','may’ve': 'may have','mayn’t': 'may not',
 'might’ve': 'might have','mightn’t': 'might not','mightn’t’ve': 'might not have',
 'must’ve': 'must have','mustn’t': 'must not','mustn’t’ve': 'must not have',
 'needn’t': 'need not','needn’t’ve': 'need not have','ne’er': 'never',
 'o’': 'of','o’clock': 'of the clock','ol’': 'old',
 'oughtn’t': 'ought not','oughtn’t’ve': 'ought not have','o’er': 'over',
 'shan’t': 'shall not','sha’n’t': 'shall not','shalln’t': 'shall not',
 'shan’t’ve': 'shall not have','she’s': 'she is','she’ll': 'she will',
 'she’d': 'she would','she’d’ve': 'she would have','should’ve': 'should have',
 'shouldn’t': 'should not','shouldn’t’ve': 'should not have','so’ve': 'so have',
 'so’s': 'so is','somebody’s': 'somebody is','someone’s': 'someone is',
 'something’s': 'something is','that’re': 'that are','that’s': 'that is',
 'that’ll': 'that will','that’d': 'that would','that’d’ve': 'that would have',
 'there’re': 'there are','there’s': 'there is','there’ll': 'there will',
 'there’d': 'there would','there’d’ve': 'there would have','these’re': 'these are',
 'they’re': 'they are','they’ve': 'they have','they’ll': 'they will',
 'they’ll’ve': 'they will have','they’d': 'they would','they’d’ve': 'they would have',
 'this’s': 'this is','those’re': 'those are','to’ve': 'to have',
 'wasn’t': 'was not','we’re': 'we are','we’ve': 'we have',
 'we’ll': 'we will','we’ll’ve': 'we will have','we’d': 'we would',
 'we’d’ve': 'we would have','weren’t': 'were not','what’re': 'what are',
 'what’d': 'what did','what’ve': 'what have','what’s': 'what is',
 'what’ll': 'what will','what’ll’ve': 'what will have','when’ve': 'when have',
 'when’s': 'when is','where’re': 'where are','where’d': 'where did',
 'where’ve': 'where have','where’s': 'where is','which’s': 'which is',
 'who’re': 'who are','who’ve': 'who have','who’s': 'who is',
 'who’ll': 'who will','who’ll’ve': 'who will have','who’d': 'who would',
 'who’d’ve': 'who would have','why’re': 'why are','why’d': 'why did',
 'why’ve': 'why have','why’s': 'why is','will’ve': 'will have',
 'won’t': 'will not','won’t’ve': 'will not have','would’ve': 'would have',
 'wouldn’t': 'would not','wouldn’t’ve': 'would not have','y’all': 'you all',
 'y’all’re': 'you all are','y’all’ve': 'you all have','y’all’d': 'you all would',
 'y’all’d’ve': 'you all would have','you’re': 'you are','you’ve': 'you have',
 'you’ll’ve': 'you shall have','you’ll': 'you will','you’d': 'you would','you’d’ve': 'you would have',
 'colour':'color','centre':'center', 'favourite':'favorite',
 'travelling':'traveling','counselling':'counseling','theatre':'theater',
 'cancelled':'canceled','labour':'labor','organisation':'organization',
 'wwii':'world war 2','citicise':'criticize','instagram': 'social medium',
 'whatsapp': 'social medium','snapchat': 'social medium', 'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 
 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 
 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize',
 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 
 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 
 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating',
 'pennis': 'penis', 'Etherium': 'bitcoin', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend',
 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization',
 'weatern':'western','interledger':'blockchain','deplation':'deflation', 'cryptocurrencies':'cryptocurrency', 'bitcoin':'blockchain cryptocurrency',
 'electroneum':'bitcoin','nanodegree':'degree','hotstar':'star','dream11':'dream','ftre':'fire','tensorflow':'framework','unocoin':'bitcoin',
 'lnmiit':'limit','unacademy':'academy','altcoin':'bitcoin','altcoins':'bitcoin','litecoin':'bitcoin','coinbase':'bitcoin','cryptocurency':'cryptocurrency',
 'simpliv':'simple','quoras':'quora','schizoids':'psychopath','remainers':'remainder','twinflame':'soulmate','quorans':'quora','brexit':'demonetized'}

def clean_misspell(word):
    try:
        x = misspell_dict[word]
    except KeyError:
        x = word
    return x

In [ ]:
def clean_stopwords(x):
  x = [word for word in x.split() if word not in nltk_stopwords]
  x = ' '.join(x)
  return x

***Tổng hợp hàm tiền xử lý***

In [ ]:
def preprocessing(x):
    x = clean_punct(x)
    x = clean_misspell(x)
    x = clean_numbers(x)
    x = clean_stopwords(x)
    return x;

***Tiền xử lý dữ liệu train***

In [ ]:
train_data['question_text_cleaned'] = train_data['question_text'].apply(preprocessing)

**VECTOR HÓA DỮ LIỆU BẰNG TF-IDF**

***Biểu diễn dữ liệu bằng TF-IDF***

TF-IDF (Term Frequency – Inverse Document Frequency) là 1 kĩ thuật sử dụng trong khai phá dữ liệu văn bản. Trọng số này được sử dụng để đánh giá tầm quan trọng của một từ trong một văn bản. Giá trị cao thể hiện độ quan trọng cao và nó phụ thuộc vào số lần từ xuất hiện trong văn bản nhưng bù lại bởi tần suất của từ đó trong tập dữ liệu.

TF: Term Frequency(Tần suất xuất hiện của từ) là số lần từ xuất hiện trong văn bản.Vì các văn bản có thể có độ dài ngắn khác nhau nên một số từ có thể xuất hiện nhiều lần trong một văn bản dài hơn là một văn bản ngắn. Như vậy, term frequency thường được chia cho độ dài văn bản( tổng số từ trong một văn bản).

IDF: Inverse Document Frequency(Nghịch đảo tần suất của văn bản), giúp đánh giá tầm quan trọng của một từ. Khi tính toán TF , tất cả các từ được coi như có độ quan trọng bằng nhau. Nhưng một số từ như “is”, “of” và “that” thường xuất hiện rất nhiều lần nhưng độ quan trọng là không cao. Như thế chúng ta cần giảm độ quan trọng của những từ này xuống.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(train_data['question_text_cleaned'].tolist())

Chia dữ liệu theo tỉ lẹ 80:20. 20% để kiểm tra và 80% để huấn luyện

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, train_data.target, test_size=0.2, random_state=1)

# CÁC PHƯƠNG PHÁP CÂN BẰNG DỮ LIỆU

Do dữ liệu mất cân bằng nên ta cần **cân bằng lại dữ liệu**. Em sẽ sử dụng Oversampling và Undersampling để cân bằng dữ liệu.

***Undersampling*** lấy mẫu làm cân bằng tập dữ liệu bằng cách giảm kích thước của lớp trội. Phương pháp này được sử dụng khi số lượng dữ liệu là đủ. Bằng cách giữ tất cả các mẫu trong lớp hiếm và chọn ngẫu nhiên một số trong lớp trội, một tập dữ liệu cân bằng mới có thể được lấy ra để lập mô hình tiếp theo.

***Oversampling*** là kỹ thuật lấy mẫu quá mức, tức là lấy ngẫu nhiên các mẫu dữ liệu ở lớp thiểu số (trong trường hợp này là các câu hỏi nhãn 0) để bù thêm vào chính nó nhằm đạt được sự phân bố số lượng xấp xỉ với lớp đa số(câu hỏi có nhãn 1).

In [ ]:
#OVERSAMPLING
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(sampling_strategy=0.6, random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [ ]:
import collections
count = collections.Counter(y_resampled)
list_value = list(count.values())
ratio = pd.DataFrame({
    'Label': ['0','1'],

    'Tỉ lệ': [list_value[0]/(list_value[0] + list_value[1]) * 100,
              list_value[1]/(list_value[0] + list_value[1]) * 100]})
ratio

In [ ]:
# UNDERSAMPLING
from imblearn.under_sampling import RandomUnderSampler
ros = RandomUnderSampler(sampling_strategy=0.4, random_state=1)
X_resampled_under, y_resampled_under = ros.fit_resample(X_train, y_train)

# ÁP DỤNG VÀO MODEL

**NAIVE BAYES**

Naive Bayes chủ yếu được sử dụng trong phân loại văn bản. Đặc trưng đầu vào ở đây chính là tần suất xuất hiện của từ trong văn bản đó.

**Naive Bayes trước khi resampling**

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model_nb = MultinomialNB().fit(X_train, y_train)
y_pred = model_nb.predict(X_test)
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("F1-score: ",f1_score(y_test, y_pred, pos_label=1))

**NB sau khi oversampling**

In [ ]:
model_nb = MultinomialNB().fit(X_resampled, y_resampled)
y_resampled_pred = model_nb.predict(X_test)
print("Accuracy: ", accuracy_score(y_test, y_resampled_pred))
print("F1-score: ",f1_score(y_test, y_resampled_pred, pos_label=1))

**NB sau khi undersampling**

In [ ]:
model_nb = MultinomialNB().fit(X_resampled_under, y_resampled_under)
y_resampled_pred = model_nb.predict(X_test)
print("Accuracy: ", accuracy_score(y_test, y_resampled_pred))
print("F1-score: ",f1_score(y_test, y_resampled_pred, pos_label=1))

***Nhận xét:*** Trước khi resampling thì kết quả F1 khá thấp do dữ liệu mất cân bằng. Nhưng sau đó F1 đã tăng 0.4 đối với undersampling và 0.4 với oversampling

**LOGISTIC REGRESSION**

**Logistic Regression** là một thuật toán phân loại dùng để gán đối tượng cho một tập giá trị rời rạc và được ứng dụng trong phân loại thư rác và bài toán phân loại câu hỏi insincere này khá tương đồng.

**LR trước khi resampling**

In [ ]:
from sklearn.linear_model import LogisticRegression
model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)

y_pred = model_lr.predict(X_test)
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("F1-score: ",f1_score(y_test, y_pred, pos_label=1))

***Nhận xét:*** LR có kết quả khá cao mặc dù dữ liệu mất cân bằng

**LR sau khi oversampling**

In [ ]:
model_lr = LogisticRegression(max_iter=300)
model_lr.fit(X_resampled, y_resampled)

y_resampled_pred = model_lr.predict(X_test)
print("Accuracy: ", accuracy_score(y_test, y_resampled_pred))
print("F1-score: ",f1_score(y_test, y_resampled_pred, pos_label=1))

**LR sau khi undersampling**

In [ ]:
model_lr = LogisticRegression(max_iter=300)
model_lr.fit(X_resampled_under, y_resampled_under)

y_resampled_pred = model_lr.predict(X_test)
print("Accuracy: ", accuracy_score(y_test, y_resampled_pred))
print("F1-score: ",f1_score(y_test, y_resampled_pred, pos_label=1))

***Nhận xét:*** Sau khi xử lý mất cân bằng thì độ đo F1 không thay đổi nhiều những kết quả sau khi xử lý với Undersampling vẫn cao nhất

# TIỀN XỬ LÝ DỮ LIỆU VỚI TẬP EMBEDDINGS

***Word Embedding*** là mô hình ngôn ngữ và các phương pháp học theo đặc trưng trong Xử lý ngôn ngữ tự nhiên (NLP), ở đó các từ hoặc cụm từ được ánh xạ sang các vector số. Thay vì đi thẳng vào xử lý dữ liệu tiêu chuẩn như bên trên em sẽ sử dụng file pretrained có sẵn của GoogleNews. Tập embedding có sẵn của GoogleNews là một tập chứa các từ và vector có sẵn của từ đó. 

Đoạn code dưới đây là để theo dõi vốn từ huấn luyện, nó sẽ đi qua tất cả các text và đếm số lần xuất hiện của các từ có trong đó. Ta sẽ xây dựng một tập từ vựng dựa trên tập có sẵn của GoogleNews

In [ ]:
from tqdm import tqdm
tqdm.pandas()
def build_vocab(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {} 
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [ ]:
sentences = train_data["question_text"].progress_apply(lambda x: x.split()).values
vocab = build_vocab(sentences)
print({k: vocab[k] for k in list(vocab)[:5]})

**Unzip tập embeddings và load file embeddings**

In [ ]:
!unzip ../input/quora-insincere-questions-classification/embeddings.zip

In [ ]:
from gensim.models import KeyedVectors

news_path = './GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'
embeddings_index = KeyedVectors.load_word2vec_format(news_path, binary=True)

Đây là hàm kiểm tra sự tương quan giữa từ vựng với embeddings, xem xem tập vocab đó bao phủ bao nhiêu phần trăm dữ liệu của mình. Output sẽ là dánh sách các từ out of vocabulary (oov) mà ta có thể sử dụng để cải thiện quá trình tiền xử lý dữ liệu


In [ ]:
import operator 

def check_coverage(vocab,embeddings_index):
    a = {}
    oov = {}
    k = 0
    i = 0
    for word in tqdm(vocab):
        try:
            a[word] = embeddings_index[word]
            k += vocab[word]
        except:

            oov[word] = vocab[word]
            i += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(a) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(k / (k + i)))
    sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]

    return sorted_x

In [ ]:
oov = check_coverage(vocab,embeddings_index)

Nhận xét: Chỉ có 24.31% từ vựng có embeddings nhưng độ bao phủ khá cao 78.75%. Ta sẽ quan sát oov đứng đầu

In [ ]:
oov[:10]

In [ ]:
'?' in embeddings_index

In [ ]:
'&' in embeddings_index

Trong khi các kí tự đặc biệt có trong Google Embeddings thì các dấu chấm câu lại không có -> ta sẽ tạo một hàm để tách các kí tự đặc biệt và xóa những kí tự còn lại

In [ ]:
def clean_puncts(x):
    x = str(x)
    for punct in "/-'":
        x = x.replace(punct, ' ')
    for punct in '& "#$%\'()*+-/ <=>@[\\]^_`{|}~':
        x = x.replace(punct, f' {punct} ')
    for punct in '?!.,:;' '“”’':
        x = x.replace(punct, '')
    return x

In [ ]:
train_data["question_text"] = train_data["question_text"].progress_apply(lambda x: clean_puncts(x))
sentences = train_data["question_text"].apply(lambda x: x.split())
vocab = build_vocab(sentences)

In [ ]:
oov = check_coverage(vocab,embeddings_index)

**Nhận xét:** kết quả tăng từ 24.31% đến 58.55% sau khi ta xử lý các kí tự đặc biệt

In [ ]:
oov[:10]

Tiếp theo, ta sẽ loại bỏ những chữ số trong dữ liệu và thay những số lớn hơn 9 bằng dấu ## qua hàm ***clean_numbers()***

In [ ]:
import re
def clean_number(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

In [ ]:
train_data["question_text"] = train_data["question_text"].progress_apply(lambda x: clean_numbers(x))
sentences = train_data["question_text"].progress_apply(lambda x: x.split())
vocab = build_vocab(sentences)

In [ ]:
oov = check_coverage(vocab,embeddings_index)

**Nhận xét:** Kết quả tăng lên 62.29% sau khi ta xử lý chữ số. Không tăng nhiều như xử lý kí tự đặc biệt nhưng cũng có cải thiện

In [ ]:
oov[:20]

Vấn đề tiếp theo là những từ tiếng anh thường được đánh vần nhầm như 'color' và 'colour'hay những từ viết tắt. Ta sẽ chuyển các từ này thành từ hoàn chỉnh để không làm mất ý của câu hỏi bằng hàm ***clean_misspell()***

In [ ]:
train_data["question_text"] = train_data["question_text"].progress_apply(lambda x: clean_misspell(x))
sentences = train_data["question_text"].progress_apply(lambda x: x.split())
vocab = build_vocab(sentences)

In [ ]:
oov = check_coverage(vocab,embeddings_index)

In [ ]:
oov[:10]

Ta sẽ loại bỏ các từ stopwords như 'a', 'of', and'...bằng hàm ***clean_stopwords()***

In [ ]:
def clean_stopword(x):
  x = [word for word in x.split() if word not in nltk_stopwords]
  x = ' '.join(x)
  return x

In [ ]:
train_data["question_text"] = train_data["question_text"].progress_apply(lambda x: clean_stopwords(x))
sentences = train_data["question_text"].progress_apply(lambda x: x.split())
vocab = build_vocab(sentences)

In [ ]:
oov = check_coverage(vocab,embeddings_index)

**Nhận xét:** Kết quả không thay đổi sau khi ta xử lý các lỗi từ vựng và stopwords nhưng độ phủ tăng lên 96.50%


In [ ]:
oov[:10]

Sau khi tiền xử lý thì những từ không xuất hiện trong embeddings bây giờ không có giá trị nên ta có thể loại bỏ chúng

In [ ]:
def clean(x):
    x = clean_puncts(x)
    x = clean_misspell(x)
    x = clean_number(x)
    x = clean_stopword(x)
    return x

In [ ]:
train_data.head()

**Build tập vocab**

In [ ]:
train_data['processed'] = train_data['question_text'].progress_map(lambda x: clean(x))

In [ ]:
train_data.head()

In [ ]:
test_data = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/test.csv')

In [ ]:
test_data.head()

In [ ]:
test_data['processed'] = test_data['question_text'].progress_map(lambda x: clean(x))

In [ ]:
text = pd.concat([train_data.processed, test_data.processed])
vocabulary = build_vocab(text)

In [ ]:
max_feat=30000
feat_vec=300
max_len=55 #toi da so tu cua cau hoi duoc dung
vocab_size=len(vocabulary)

Sau đây, ta sẽ lấy ra index của tập từ điển để mã hóa one hot

In [ ]:
def get_word_index(vocab): #lấy ra index của từ trong tập từ vựng
    word_index=dict((w,i+1) for i,w in enumerate(vocab.keys()))
    return word_index
def fit_one_hot(word_index,corpus): #mã hóa 
    sent=[]
    for text in tqdm(corpus):
        li=[]
        for word in text.split():
            try:
                li.append(word_index[word])
            except KeyError:
                li.append(0)
        sent.append(li)
    return sent

Chia tập dữ liệu train thành train:validation theo tỉ lệ 4:1

In [ ]:
from sklearn.model_selection import train_test_split
train, val=train_test_split(train_data,test_size=0.2,stratify=train_data.target,random_state=123)

In [ ]:

word_index=get_word_index(vocabulary)
### Chuẩn bị dữ liệu đã được xử lý
train_text = train['processed']
val_text = val['processed']
test_text = test_data['processed']

### mã hóa câu trong tập train sang dạng onehot cho dễ xử lý
encodes=fit_one_hot(word_index,train_text)
train_padded=pad_sequences(encodes,maxlen=max_len,padding="post")

### mã hóa câu trong tập validation sang dạng onehot cho dễ xử lý
encodes_=fit_one_hot(word_index,val_text)
val_padded=pad_sequences(encodes_,maxlen=max_len,padding="post")

### mã hóa câu trong tập test sang dạng onehot cho dễ xử lý
encodes__=fit_one_hot(word_index,test_text)
test_padded=pad_sequences(encodes__,maxlen=max_len,padding="post")

In [ ]:
count=0
#Khởi tạo ma trận embeddings với size của tập vocab
embedding_mat=np.zeros((vocab_size,300))
for word,i in tqdm(word_index.items()):
    try:
        vec=embeddings_index[word]
        embedding_mat[i]=vec
    except KeyError:
        count+=1
        continue

print("Number of Out of Vocabulary",count)

In [ ]:
test_padded.shape

In [ ]:
train_padded.shape

# ÁP DỤNG VÀO MÔ HÌNH LOGISTIC REGRESSION

Logistic Regression là một thuật toán phân loại dùng để gán đối tượng cho một tập giá trị rời rạc và được ứng dụng trong phân loại thư rác và bài toán phân loại câu hỏi insincere này khá tương đồng.

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import *

In [ ]:

%%time
model = LogisticRegression(max_iter=300)
## Chạy mô hình
print(f"Running Logistic Regression")
model.fit(train_padded, train.target)


y_pred = model_lr.predict(val_padded)
print("Accuracy: ", accuracy_score(val.target, y_pred))
print("F1-score: ",f1_score(val.target, y_pred, pos_label=1))